# Algorithms (pt. 2)



**Exercise:** Review list of standard algorithms.

**Exercise:** Find a _raw loop_ in the ZString implementation. Claim it on the wiki https://git.corp.adobe.com/better-code/class/wiki/class-04-registration. Improve the code, create a pull-request, and assign me as the reviewer. The PR should include a http://quick-bench.com/ benchmark of the relevant code for comparison.

In [1]:
#include <algorithm>
#include <cstdint>
#include <iostream>
#include <iterator>

using namespace std;

In [2]:
namespace {

struct ZString {
    void DataAndLength(const char16_t*& data, uint32_t& length) const {
        static const char16_t r[] = u"Unicode String© with : replacements";
        data = &r[0];
        length = size(r);
    }

    auto range() const {
        static const char16_t r[] = u"Unicode String© with : replacements";
        return make_pair(begin(r), end(r));
    }

    void ReplaceCharacters(const char16_t* badChars,
                           const char16_t* replacementChars,
                           uint32_t numChars);
};

struct ARawZString {
    unique_ptr<char16_t[]> _data;

    ARawZString(uint32_t n) : _data(new char16_t[n]) {}
    char16_t* get() const { return _data.get(); }
    char16_t& operator[](size_t n) { return _data[n]; }
};

void EquateToUnicodeString(ZString&, const char16_t* p, size_t n) {
    copy_n(p, n, ostream_iterator<char>(cout, ""));
    cout << endl;
}

} // namespace

In [3]:
void ZString::ReplaceCharacters(const char16_t* badChars,
                                const char16_t* replacementChars,
                                uint32_t numChars) {
    const char16_t* myData = nullptr;
    uint32_t myLength = 0;
    DataAndLength(myData, myLength);

    bool hasBadCharacters = false;
    {
        for (uint32_t i = 0; i < myLength; ++i) {
            for (uint32_t j = 0; j < numChars; ++j) {
                if (myData[i] == badChars[j]) {
                    hasBadCharacters = true;
                    break;
                }
            }

            if (hasBadCharacters) break;
        }
    }

    if (hasBadCharacters) {
        uint32_t destLength = myLength;
        ARawZString destData(destLength);

        for (uint32_t i = 0; i < myLength; ++i) {
            destData[i] = myData[i];

            for (uint32_t j = 0; j < numChars; ++j) {
                if (myData[i] == badChars[j]) {
                    destData[i] = replacementChars[j];
                    break;
                }
            }
        }

        EquateToUnicodeString(*this, destData.get(), destLength);
    }
}

In [4]:
.undo 1

In [5]:
void ZString::ReplaceCharacters(const char16_t* badChars,
                                const char16_t* replacementChars,
                                uint32_t numChars) {
    const char16_t* myData = nullptr;
    uint32_t myLength = 0;
    DataAndLength(myData, myLength);

    auto l = myData + myLength;
    bool hasBadCharacters = any_of(myData, l, [&](auto c) {
        auto l = badChars + numChars;
        return find(badChars, l, c) != l; // NOTE: Won't scale
    });

    if (hasBadCharacters) {
        uint32_t destLength = myLength;
        ARawZString destData(destLength);

        transform(myData, l, destData.get(), [&](auto c) {
            auto l = badChars + numChars;
            auto p = find(badChars, l, c);
            if (p != l) return replacementChars[p - badChars];
            return c;
        });

        EquateToUnicodeString(*this, destData.get(), destLength);
    }
}

In [6]:
.undo 1

In [7]:
void ZString::ReplaceCharacters(const char16_t* badChars,
                                const char16_t* replacementChars,
                                uint32_t numChars) {

    auto fb = badChars, lb = badChars + numChars;

    auto [f, l] = range();

    auto p = find_if(f, l, [&](auto c) {
        return find(fb, lb, c) != lb; // NOTE: Won't scale
    });

    if (p == l) return;

    u16string result;
    result.reserve(l - f);
    result.append(f, p);

    transform(p, l, back_inserter(result), [&](auto c) {
        auto p = find(fb, lb, c);
        if (p == lb) return c;
        return replacementChars[p - fb];
    });

    EquateToUnicodeString(*this, result.data(), result.size());
}


In [8]:
{
    ZString example;
    example.ReplaceCharacters(u":©", u"/c", 2);
}

Unicode Stringc with / replacements 


## More advanced algorithms

### Sorting

- `sort()`
- `stable_sort()`

### Algorithms requiring a sorted sequence

- `lower_bound()`
- `upper_bound()`
- `equal_range()`
- `merge()`
- `inplace_merge()`
- `includes()`
- `set_difference()`
- `set_intersection()`
- `set_symmetric_difference()`
- `set_union()`

## New Algorithms (C++11 - 20)

## Projection Functions

## Position Based Algorithms
  - All non-modifying sequence operations taking a predicate
  
## Strict Weak Order

## Iterator hierarchy (and why you probably shouldn't care)

## Writing a custom algorithm
- what to return

## Composition vs. multi-pass

## Generators vs input iterator

## Output iterators vs sink functions